#### 1. IMPORT LIBRARIES AND CIFAR 10

In [19]:
# general
import numpy as np
import keras
import os
import time
import matplotlib.pyplot as plt
% matplotlib inline

# dataset
from keras.datasets import cifar10

# modeling tools
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam
from keras.utils.np_utils import to_categorical

Check and Prepare Data:

In [9]:
# split data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
# check data dimension
print('training data shape: {}'.format(x_train.shape))
print('test data shape: {}'.format(x_test.shape))
print('shape of one instance: {}'.format(x_train.shape[1:]))

training data shape: (50000, 32, 32, 3)
test data shape: (10000, 32, 32, 3)
shape of one instance: (32, 32, 3)


In [10]:
# check labels
labels = []
for y in y_train.flatten():
    if y not in labels:
        labels.append(y)
print('training labels are: {}'.format(labels))
print('# labels: {}'.format(len(labels)))

training labels are: [6, 9, 4, 1, 2, 7, 8, 3, 5, 0]
# labels: 10


In [11]:
cifar_classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [12]:
# Convert class vectors to one-hot encoded vectors.
y_train = to_categorical(y_train, num_classes = 10)
y_test = to_categorical(y_test, num_classes = 10)

In [13]:
# # Transform images from (32,32,3) to 3072-dimensional vectors (32*32*3)

# x_train = np.reshape(x_train,(50000,3072))
# x_test = np.reshape(x_test,(10000,3072))
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalization of pixel values (to [0-1] range)

x_train /= 255
x_test /= 255

#### 2. HELP FUNCTION

In [14]:
def plotAcc(history):
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model_accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc = 'upper right')
    
def plotLoss(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model_loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc = 'upper right')

#### 3. BUILD CNN CLASSIFIER

In [17]:
num_classes= 10

##### RMSprop - LR = 0.0001

In [20]:
# hyper parameters to be tuned
n_neuron = 256
batch_size = 32
dropout_rate = 0.2
data_augmentation = True
epochs = 100
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# set up model structure
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
    
# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)
 
    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)


# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Using real-time data augmentation.
Epoch 1/100
1563/1563 [==============================] - 253s 162ms/step - loss: 1.8749 - acc: 0.3083 - val_loss: 1.5650 - val_acc: 0.4298
Epoch 2/100
1563/1563 [==============================] - 245s 157ms/step - loss: 1.5941 - acc: 0.4188 - val_loss: 1.4158 - val_acc: 0.4894
Epoch 3/100
1563/1563 [==============================] - 245s 156ms/step - loss: 1.4657 - acc: 0.4690 - val_loss: 1.4036 - val_acc: 0.4968
Epoch 4/100
1563/1563 [==============================] - 230s 147ms/step - loss: 1.3819 - acc: 0.5057 - val_loss: 1.2733 - val_acc: 0.5404
Epoch 5/100
1563/1563 [==============================] - 248s 158ms/step - loss: 1.3113 - acc: 0.5358 - val_loss: 1.1206 - val_acc: 0.6031
Epoch 6/100
1563/1563 [==============================] - 318s 203ms/step - loss: 1.2460 - acc: 0.5551 - val_loss: 1.1075 - val_acc: 0.6033
Epoch 7/100
1563/1563 [==============================] - 442s 283ms/step - loss: 1.1946 - acc: 0.5773 - val_loss: 1.0258 - val_acc:

1563/1563 [==============================] - 246s 158ms/step - loss: 0.7968 - acc: 0.7329 - val_loss: 0.7888 - val_acc: 0.7361
Epoch 60/100
1563/1563 [==============================] - 255s 163ms/step - loss: 0.7984 - acc: 0.7317 - val_loss: 0.7397 - val_acc: 0.7614
Epoch 61/100
1563/1563 [==============================] - 253s 162ms/step - loss: 0.7898 - acc: 0.7334 - val_loss: 0.6777 - val_acc: 0.7791
Epoch 62/100
1563/1563 [==============================] - 261s 167ms/step - loss: 0.7980 - acc: 0.7318 - val_loss: 0.6800 - val_acc: 0.7769
Epoch 63/100
1563/1563 [==============================] - 251s 161ms/step - loss: 0.7923 - acc: 0.7358 - val_loss: 0.6914 - val_acc: 0.7689
Epoch 64/100
1563/1563 [==============================] - 266s 170ms/step - loss: 0.7947 - acc: 0.7318 - val_loss: 0.7679 - val_acc: 0.7570
Epoch 65/100
1563/1563 [==============================] - 275s 176ms/step - loss: 0.7958 - acc: 0.7337 - val_loss: 0.7599 - val_acc: 0.7526
Epoch 66/100
1563/1563 [=========

##### Adam - LR = 0.0001

In [21]:
# hyper parameters to be tuned
n_neuron = 256
batch_size = 32
dropout_rate = 0.2
data_augmentation = True
epochs = 100
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models_2')
model_name = 'keras_cifar10_trained_model_2.h5'

# set up model structure
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
  
# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)
 
    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)


# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Using real-time data augmentation.
Epoch 1/100
1563/1563 [==============================] - 214s 137ms/step - loss: 1.8261 - acc: 0.3256 - val_loss: 1.5104 - val_acc: 0.4460
Epoch 2/100
1563/1563 [==============================] - 225s 144ms/step - loss: 1.5427 - acc: 0.4360 - val_loss: 1.3955 - val_acc: 0.4963
Epoch 3/100
1563/1563 [==============================] - 218s 139ms/step - loss: 1.4372 - acc: 0.4768 - val_loss: 1.2579 - val_acc: 0.5469
Epoch 4/100
1563/1563 [==============================] - 224s 143ms/step - loss: 1.3584 - acc: 0.5105 - val_loss: 1.1869 - val_acc: 0.5759
Epoch 5/100
1563/1563 [==============================] - 218s 139ms/step - loss: 1.2845 - acc: 0.5399 - val_loss: 1.1670 - val_acc: 0.5815
Epoch 6/100
1563/1563 [==============================] - 218s 140ms/step - loss: 1.2347 - acc: 0.5615 - val_loss: 1.0591 - val_acc: 0.6219
Epoch 7/100
1563/1563 [==============================] - 216s 138ms/step - loss: 1.1781 - acc: 0.5806 - val_loss: 1.0431 - val_acc:

1563/1563 [==============================] - 211s 135ms/step - loss: 0.5916 - acc: 0.7904 - val_loss: 0.5454 - val_acc: 0.8128
Epoch 60/100
1563/1563 [==============================] - 230s 147ms/step - loss: 0.5885 - acc: 0.7943 - val_loss: 0.5365 - val_acc: 0.8163
Epoch 61/100
1563/1563 [==============================] - 230s 147ms/step - loss: 0.5864 - acc: 0.7951 - val_loss: 0.5394 - val_acc: 0.8144
Epoch 62/100
1563/1563 [==============================] - 219s 140ms/step - loss: 0.5851 - acc: 0.7961 - val_loss: 0.5460 - val_acc: 0.8116
Epoch 63/100
1563/1563 [==============================] - 220s 141ms/step - loss: 0.5780 - acc: 0.7991 - val_loss: 0.5310 - val_acc: 0.8177
Epoch 64/100
1563/1563 [==============================] - 225s 144ms/step - loss: 0.5762 - acc: 0.7978 - val_loss: 0.5199 - val_acc: 0.8206
Epoch 65/100
1563/1563 [==============================] - 214s 137ms/step - loss: 0.5693 - acc: 0.8011 - val_loss: 0.5313 - val_acc: 0.8173
Epoch 66/100
1563/1563 [=========